<a href="https://colab.research.google.com/github/HAMZA310/-CS231n-Convolutional-Neural-Networks-for-Visual-Recognition-STANFORD/blob/master/week_2__first_iteration_Q_learning_code_from_NP_to_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### TO DO:
`Make the minium possible changes to convert to PyTorch. No premature optimizations. No vectorizations. Let the loops stay. All optimizations in the next interations.`

### Notes
- Write unit tests to make sure each tensor equivalent is exactly equivalent to Numpy Operation.
- Unit tests on checking shapes are also very important.
- See if loops are avoidable. For example, bmm (batch mat mul is at your disposal).
- Check if converting scalars to tensors provides a speedup? Checked. Not noticeable. But it's not a bad idea to put everything in a tensor.


### From coder
The important test case is the CartPole environment (you can search the colab document for that). If you want to have more low-level control, the relevant case is when the methods add_point, update_Q, estimate Q are called repeatedly in a loop for as long as is feasible. For the add_point method, you can set s, a, r, and s' however you like, as long as s and s' always have the same dimension, a is in {0, ..., self.A - 1}, and r is in [0, 1].


We haven't yet written the theoretical resource this code is most precisely based on. However, what's being encoded here is using [Guassian processes](http://mlg.eng.cam.ac.uk/teaching/4f13/1920/gp%20and%20data.pdf) to estimate a [finite-horizon](https://arxiv.org/pdf/1909.03906.pdf) version of the Q-value of a state-action pair in a Markov Decision Process. Q_S[depth] contains estimates of the expectation of the average reward over the next `depth` timesteps after starting in state s and taking action a, for all (s, a) pairs we have observed so far. Then the method pictured above comes up with an estimate for an arbitrary state-action pair.


In [13]:
import torch
print(torch.__version__)

_device = 'cpu' # or 'cuda

1.8.1+cu101


In [14]:
import numpy as np
from scipy.special import betaincinv, erfinv
import matplotlib.pyplot as plt

In [15]:
# No Numpy code in this cell

import time
class Check():
    def __init__(self):
        self.checkpoint_dict = dict()
        self.last_check = None
        self.last_time = None
    def check(self, checkpoint_name):
        now = time.time()
        if self.last_check is not None:
            interval_name = self.last_check + " -> " + checkpoint_name
            if interval_name not in self.checkpoint_dict:
                self.checkpoint_dict[interval_name] = []
            self.checkpoint_dict[interval_name].append(now - self.last_time)
        self.last_check = checkpoint_name
        self.last_time = now
    def print_total_times(self):
        for interval in self.checkpoint_dict:
            print(interval, ":", sum(self.checkpoint_dict[interval]))
    def reset(self):
        self.__init__()
check = Check()

In [16]:
# torch Tensors Version
import matplotlib.tri as mtri
from mpl_toolkits.mplot3d import Axes3D

def tensor_array(arr, shape_if_empty):
    #############   FIX NEEDED ##########
    ### TO DO: should return a Tensor not a list
    #############   FIX NEEDED ##########


    "arr is of type list"
    if len(arr) == 0:
        return [torch.tensor([]).reshape(shape_if_empty)]

    # tensor = torch.tensor(arr)
    # print(f'after conversion got {tensor}')
    # return arr
    return [elm.reshape(1,-1) for elm in arr] # This is reshaped to a higher dimension in np version

def tensor_concat(pair):
    "pair is of type 'tuple'/'pair'"
    if len(pair[0]) == 0:
        return torch.tensor([pair[1]])
    else:
        return torch.cat(pair)

class Tn_QueueQGP(object):
    def __init__(
        self, n_actions, kernel, kernel_batch, 
        states, actions, rewards, next_states, 
        dones, done_value=0, optimistic=True, 
        bias_count=1, float_type=torch.float32, 
        advantage_learning=True
        ):
        "Assume that if it's array like, then it's a tensor"
        "Assume scalars are Python ints or floats for now"

        # TO DO: Convert all to tensors. using `torch.tensor(x)`
        self.A = n_actions
        self.k = kernel
        self.k_batch = kernel_batch
        self.done_value = done_value
        self.float_type = float_type
        self.advantage_learning = advantage_learning

        # Vars below are array-like mostly.
        self.raw_S = states.to(float_type) 
        self.raw_S_ = next_states.to(float_type)
        self.raw_R = rewards.to(float_type) 
        self.actions = actions


        self.S = [self.raw_S[actions == a] for a in range(self.A)]
        self.R = [self.raw_R[actions == a] for a in range(self.A)]
        self.S_ = [self.raw_S_[actions == a] for a in range(self.A)]
        self.dones = [dones[actions == a] for a in range(self.A)]
        self.ns = [self.S[a].shape[0] for a in range(self.A)]
        print(self.ns)
        self.dim = states.shape[1]

        self.max_var = 1/4 # 1/4 is the maximum variance of a distribution over [0, 1] # check float length compatibility
        self.optimistic = optimistic
        if self.optimistic:
            self.baseline = 1
        else:
            self.baseline = 0
        self.bias_count = bias_count
        self.data_noise = self.max_var * self.bias_count
        self.K = []
        
        # These nested for loops should be replaced with tensor operations.
        for a in range(self.A):
            self.K.append(torch.eye(self.ns[a]) * self.data_noise)
            for i in range(self.ns[a]):
                for j in range(i + 1):
                    if i == j:
                        assert(k(self.S[a][i], self.S[a][j]) == self.max_var)
                    self.K[a][i][j] += k(self.S[a][i], self.S[a][j])
                    self.K[a][j][i] = self.K[a][i][j]
        
        self.K_inv = [torch.linalg.pinv(self.K[a]) for a in range(self.A)]
        self.K_SS_ = [
                      [tensor_array(
                          [self.K_S(s_, a_) for s_ in self.S_[a]]
                          , shape_if_empty=(0, self.ns[a_])
                          )
                      for a_ in range(self.A)]
                      for a in range(self.A)
                      ] # indexed as K_SS_[action that s_ followed][action following s_][s_][s]
         
        self.w_SS_ = [
                      [torch.mm(self.K_SS_[a][a_][0], self.K_inv[a_])
                       for a_ in range(self.A)] 
                      for a in range(self.A)
                      ] # indexed as w_SS_[action that s_ followed][action of interest][s_][s]
        self.var_S_ = [[torch.tensor([self.max_var - self.w_SS_[a][a_][s_] * self.K_SS_[a][a_][s_]\
                         if (self.w_SS_[a][a_][s_] * self.K_SS_[a][a_][s_]).numel() > 0\
                         else self.max_var - torch.zeros(1)\
                         for s_ in range(len(self.S_[a]))]) for a_ in range(self.A)]
                       for a in range(self.A)]
        
        if self.advantage_learning:
            self.K_S_S__all = torch.tensor([self.k_batch(s, self.raw_S_) for s in self.raw_S_]) + self.data_noise * torch.eye(len(self.raw_S_))

            self.K_inv_S_S__all = torch.linalg.pinv(self.K_S_S__all)

            self.K_SS_all = torch.tensor([self.k_scaled(s, self.raw_S) for s in self.raw_S]) + self.data_noise * torch.eye(len(self.raw_S))
            self.K_inv_SS_all = torch.linalg.pinv(self.K_SS_all)

        self.Q_S = None
        self.Q_S_ = None
        self.V_S = None
        self.V_S__all = None

    def K_S(self, s, a):
        return self.k_batch(s, self.S[a])
        
    def k_scaled(self, s_1, s_2):
        min_S = torch.min(self.raw_S, axis=0).values
        range_S = torch.max(self.raw_S, axis=0).values - min_S
        return self.k(range_S * (s_1 - min_S), range_S * (s_2 - min_S))

    def w(self, s, a):
        # TO DO: Replaced @ with *. Verify if matrix multiplication is required.
        return self.K_inv[a] * self.K_S(s, a)


    def add_data_point(self, s, a, r, s_, done):
        s = s.to(self.float_type)
        r = r.to(self.float_type)
        s_ = s_.to(self.float_type)

        check.check("a")
        K_Ss = self.K_S(s, a)
        # assert(not np.any(np.isnan(K_Ss)))
        # check_K_s_S_ = np.array([self.k_scaled(s_, s__) for s__ in self.raw_S_])
        if self.advantage_learning:
            K_s_S_ = self.k_batch(s_, self.raw_S_)
            # assert(np.all(np.isclose(check_K_s_S_, K_s_S_)))
            K_sS_all = self.k_batch(s, self.raw_S)

        self.raw_S = torch.vstack((self.raw_S, s.reshape(1, -1)))
        self.raw_S_ = torch.vstack((self.raw_S_, s_.reshape(1, -1)))
        self.raw_R = torch.cat((self.raw_R, torch.tensor([r])))
        self.actions = torch.cat((self.actions, torch.tensor([a])))
        self.S[a] = torch.vstack((self.S[a], s.reshape(1, -1)))
        self.S_[a] = torch.vstack((self.S_[a], s_.reshape(1, -1)))
        self.R[a] = torch.cat((self.R[a], torch.tensor([r])))
        self.dones[a] = torch.cat((self.dones[a], torch.tensor([done])))
        self.ns[a] += 1
        
        # assert(self.k_scaled(s, s) == self.max_var)
        if len(self.K[a]) == 0:
            self.K[a] = torch.tensor([[self.max_var + self.data_noise]])
        else:
            self.K[a] = torch.hstack((tensor_concat((self.K[a], K_Ss.reshape(1, -1)))\
                                      , tensor_concat((K_Ss, torch.tensor([self.max_var + self.data_noise]))).reshape(-1, 1)))

        check.check("b")
        K_inv_K_Ss = self.K_inv[a] @ K_Ss
        K_inv_ss = torch.tensor([1 / (self.max_var + self.data_noise - K_inv_K_Ss @ K_Ss)])
        self.K_inv[a] = self.K_inv[a] + K_inv_ss * torch.outer(K_inv_K_Ss, K_inv_K_Ss)
        K_inv_K_Ss_K_inv_ss = - K_inv_ss * K_inv_K_Ss
        self.K_inv[a] = torch.hstack((self.K_inv[a], K_inv_K_Ss_K_inv_ss.reshape(-1, 1)))
        self.K_inv[a] = torch.vstack((self.K_inv[a], torch.cat((K_inv_K_Ss_K_inv_ss, K_inv_ss)).reshape(1, -1)))
        check.check("c")

        if self.advantage_learning:
            if len(self.K_S_S__all) == 0:
                self.K_S_S__all = torch.tensor([[self.max_var + self.data_noise]])
            else:
                self.K_S_S__all = torch.hstack((tensor_concat((
                    self.K_S_S__all, K_s_S_.reshape(1, -1)))\
                    , tensor_concat((K_s_S_, torch.tensor([self.max_var + self.data_noise]))).reshape(-1, 1)))
            
            K_inv_S_S__K_s_S_ = self.K_inv_S_S__all @ K_s_S_
            K_inv_s_s_ = torch.tensor([1 / (self.max_var + self.data_noise - K_inv_S_S__K_s_S_ @ K_s_S_)])
            self.K_inv_S_S__all = self.K_inv_S_S__all + K_inv_s_s_ * torch.outer(K_inv_S_S__K_s_S_, K_inv_S_S__K_s_S_)
            K_inv_s_s__K_inv_S_S__K_s_S_ = - K_inv_s_s_ * K_inv_S_S__K_s_S_
            self.K_inv_S_S__all = torch.hstack((self.K_inv_S_S__all, K_inv_s_s__K_inv_S_S__K_s_S_.reshape(-1, 1)))
            self.K_inv_S_S__all = torch.vstack((self.K_inv_S_S__all, torch.cat((K_inv_s_s__K_inv_S_S__K_s_S_, K_inv_s_s_)).reshape(1, -1)))

            if len(self.K_SS_all) == 0:
                self.K_SS_all = torch.tensor([[self.max_var + self.data_noise]])
            else:
                self.K_SS_all = torch.hstack((tensor_concat(
                    (self.K_SS_all, K_sS_all.reshape(1, -1)))\
                    , tensor_concat((K_sS_all, torch.tensor([self.max_var + self.data_noise]))).reshape(-1, 1)))
            
            K_inv_SS_K_sS = self.K_inv_SS_all @ K_sS_all
            K_inv_ss_all = torch.tensor([1 / (self.max_var + self.data_noise - K_inv_SS_K_sS @ K_sS_all)])
            self.K_inv_SS_all = self.K_inv_SS_all + K_inv_ss_all * torch.outer(K_inv_SS_K_sS, K_inv_SS_K_sS)
            K_inv_ss_K_inv_SS_K_sS = - K_inv_ss_all * K_inv_SS_K_sS
            self.K_inv_SS_all = torch.hstack((self.K_inv_SS_all, K_inv_ss_K_inv_SS_K_sS.reshape(-1, 1)))
            self.K_inv_SS_all = torch.vstack((self.K_inv_SS_all, torch.cat((K_inv_ss_K_inv_SS_K_sS, K_inv_ss_all)).reshape(1, -1)))
        for a__ in range(self.A):
###################################################
                    ######################################################################################################
                    ######################################################################################################
                    ###################################################

            print(
                'self.K_SS_[a__][a]', self.K_SS_[a__][a],
                'K_inv_K_Ss', K_inv_K_Ss
            )
            self.w_SS_[a__][a] += K_inv_ss * torch.outer(
                torch.mm(self.K_SS_[a__][a], K_inv_K_Ss)\
                ,K_inv_K_Ss
                ) # because of change to self.K_inv[a][:-1, :-1]
            if a__ != a:
                ### TO DO: fix code duplication below
                print('could be empty', self.K_SS_[a__][a])
                is_kss_empty = ((self.K_SS_[a__][a])[0]).numel() == 0
                
                if not is_kss_empty:
                    print('>>>> ')
                    print(
                        self.K_SS_[a__][a],
                        self.k(s_, s)
                    )
                    
                    self.K_SS_[a__][a] = torch.hstack((torch.tensor(self.K_SS_[a__][a]), torch.tensor([self.k(s_, s)\
                                                                                        for s_ in self.S_[a__]]).reshape(-1, 1)))
                    print(">> 1observe elements", self.K_SS_)
                else:
                    self.K_SS_[a__][a] = torch.hstack((torch.tensor([]), torch.tensor([self.k(s_, s)\
                                                                                        for s_ in self.S_[a__]]).reshape(-1, 1)))
                    print(">> 2observe elements", self.K_SS_)
            else:
                print('>>>> Else')
                print(
                    self.K_SS_[a__][a],
                    self.k(s_, s)
                )

                is_kss_empty = ((self.K_SS_[a__][a])[0]).numel() == 0
                if not is_kss_empty:
                    self.K_SS_[a__][a] = torch.hstack((torch.tensor(self.K_SS_[a__][a]), torch.tensor([self.k(s_, s)\
                                                                                        for s_ in self.S_[a__][:-1]]).reshape(-1, 1)))
                    print(">> 3observe elements", self.K_SS_)
                else:
                    self.K_SS_[a__][a] = torch.hstack((torch.tensor([]), torch.tensor([self.k(s_, s)\
                                                                                        for s_ in self.S_[a__][:-1]]).reshape(-1, 1)))
                                                                                        
                    print(">> 4observe elements", self.K_SS_)
            self.w_SS_[a__][a] += torch.outer(self.K_SS_[a__][a][:, -1], K_inv_K_Ss_K_inv_ss) # because of change to last column of self.K_SS_[a__][a]
            self.w_SS_[a__][a] = torch.hstack(
                self.w_SS_[a__][a]
                , torch.mm(self.K_SS_[a__][a], self.K_inv[a][:,-1]).reshape(-1, 1)
                ) # because of extra column in self.K_inv[a]
        print(">> out observe elements", self.K_SS_)
        for a_ in range(self.A):
            elm = self.K_SS_[a][a_]
            # TO DO: You shoudn't need this hack i.e. elm[0].
            print(
                self.K_SS_[a][a_], '\n', 
                elm, '<elm\n',
                self.w_SS_[a][a_]
                )
            if isinstance(elm, list):
                self.K_SS_[a][a_] = torch.vstack(((elm[0]), (self.K_S(s_, a_))))
                self.w_SS_[a][a_] = torch.vstack((self.w_SS_[a][a_], (elm[0][-1] * elf.K_inv[a_]).reshape(1, -1)))
            else:
                self.K_SS_[a][a_] = torch.vstack(((elm), (self.K_S(s_, a_))))
                self.w_SS_[a][a_] = torch.vstack((self.w_SS_[a][a_], (elm[-1] * self.K_inv[a_]).reshape(1, -1)))
        check.check("d")

    def update_Q(self, depth, verbose=False, visualization_dimension=[0, 1]):
        check.check("f")
        if not self.advantage_learning:
            self.Q_S = [[(self.R[a] - self.baseline) - self.data_noise * self.K_inv[a] @ (self.R[a] - self.baseline) + self.baseline\
                         for a in range(self.A)]] # indexed as Q_S[depth][action][state]
        else:
            # TO DO: replace elment-wise product with matrix mulitplication? Which is requried here? Matrix Multiplication is more likely.
            IRE_S_raw = (torch.eye(self.raw_S.shape[0]) - self.data_noise * self.K_inv_SS_all) * (self.raw_R - self.baseline)\
                        + self.baseline  # I - self.data_noise * self.K_inv_SS_all = (self.K_SS_all - self.data_noise) * self.K_inv_SS_all
            
            x = self.K_inv_SS_all * (self.raw_R - self.baseline)
            IRE_S__raw = torch.tensor([self.k_batch(s_, self.raw_S) * x + self.baseline for s_ in self.raw_S_]) # redoing lots of kernel evaluations
            
            
            if verbose:
                for IRE, data in zip([IRE_S_raw, IRE_S__raw], [self.raw_S, self.raw_S_]):
                    triang = mtri.Triangulation(data[:, 0], data[:, 1])
                    fig = plt.figure()
                    ax = fig.add_subplot(1,1,1, projection='3d')
                    ax.plot_trisurf(triang, IRE, cmap='jet')
                    ax.scatter(data[:, 0],data[:, 1],IRE, marker='.', s=10, c="black", alpha=0.5)
                    ax.view_init(elev=60, azim=-45)
                    ax.set_title("IRE")
                    plt.show()
                    plt.close()
            IRE_S = [[] for a in range(self.A)]
            IRE_S_ = [[] for a in range(self.A)]
            for i in range(self.raw_S.shape[0]):
                IRE_S[self.actions[i]].append(IRE_S_raw[i])
                IRE_S_[self.actions[i]].append(IRE_S__raw[i])
            IRE_S = [torch.tensor(arr) for arr in IRE_S]
            IRE_S_ = [torch.tensor(arr) for arr in IRE_S_]

           
            self.Q_S = [[torch.tensor([self.w(s, a) * (self.R[a] - ire_s)\
                                       for s, ire_s in zip(self.S[a], IRE_S[a])]) + IRE_S[a]\
                         for a in range(self.A)]] # indexed as Q_S[depth][action][state]
            
        check.check("g")
        if self.advantage_learning:
            print('in adv learning')
            self.Q_S_ = [[[IRE_S_[a] + self.w_SS_[a][a_] * (self.Q_S[-1][a_] - IRE_S[a_])\
                           if (self.w_SS_[a][a_] * (self.Q_S[-1][a_] - IRE_S[a_])).numel() > 0\
                           else IRE_S_[a] + torch.zeros(IRE_S_[a].shape)\
                           for a_ in range(self.A)]\
                          for a in range(self.A)]] # indexed as Q_S_[depth][action][next action][next state]
        else:
            print('els adv learning')
            self.Q_S_ = [[[self.baseline + self.w_SS_[a][a_] * (self.Q_S[-1][a_] - self.baseline) for a_ in range(self.A)] for a in range(self.A)]] # indexed as Q_S_[depth][action][next action][next state]
       

        if verbose and self.advantage_learning:
            fig = plt.figure()
            ax = fig.add_subplot(1,1,1, projection='3d')
            for a in range(self.A):
                triang = mtri.Triangulation(self.S[a][:, 0],self.S[a][:, 1])
                ax.plot_trisurf(triang, self.Q_S[-1][a] - IRE_S[a], cmap='jet')
                if a == 0:
                    color = "black"
                if a == 1:
                    color = "red"
                ax.scatter(self.S[a][:, 0],self.S[a][:, 1],self.Q_S[-1][a] - IRE_S[a], marker='.', s=10, c=color, alpha=0.5)
                ax.set_title("Q_S[0] - IRE (on data)")
            ax.view_init(elev=10, azim=-45)
            plt.show()
            plt.close()

            fig = plt.figure()
            ax = fig.add_subplot(1,1,1, projection='3d')
            for a_ in range(self.A):
                X = torch.vstack([self.S_[a] for a in range(self.A)])
                y = torch.cat([self.Q_S_[-1][a][a_] for a in range(self.A)]) - torch.cat([IRE_S_[a] for a in range(self.A)])
                triang = mtri.Triangulation(X[:, 0],X[:, 1])
                ax.plot_trisurf(triang, y, cmap='jet')
                if a_ == 0:
                    color = "black"
                if a_ == 1:
                    color = "red"
                ax.scatter(X[:, 0],X[:, 1], y, marker='.', s=10, c=color, alpha=0.5)
                ax.set_title("Q_S_[0] - IRE (on and off data)")
            ax.view_init(elev=10, azim=-45)
            plt.show()
            plt.close()

            fig = plt.figure()
            ax = fig.add_subplot(1,1,1, projection='3d')
            for a_ in range(self.A):
                X = torch.vstack([self.S_[a] for a in range(self.A)])
                y = torch.cat([self.Q_S_[-1][a][a_] for a in range(self.A)])
                triang = mtri.Triangulation(X[:, 0],X[:, 1])
                ax.plot_trisurf(triang, y, cmap='jet')
                if a_ == 0:
                    color = "black"
                if a_ == 1:
                    color = "red"
                ax.scatter(X[:, 0],X[:, 1], y, marker='.', s=10, c=color, alpha=0.5)
                ax.set_title("Q_S_[0]")
            ax.view_init(elev=10, azim=-45)
            plt.show()
            plt.close()

        if self.advantage_learning:
            self.V_S_estimate = [IRE_S] # indexed V_S_estimate[depth][action following the state (just since that's how states are grouped)][state]
            self.V_S__all = [IRE_S__raw] # indexed V_S__all[depth][state s_]
        check.check("h")
        for itr, dep in enumerate(range(1, depth)):
            V_S_ = [torch.tensor(self.Q_S_[-1][a]).max(axis=0).values
                    if all(map(lambda elm: elm.numel() > 0, self.Q_S_[-1][a]))# If all are empty tensors, then max is also empty 
                    else torch.tensor([])\
                    for a in range(self.A)] # takes the maximum over next actions; indexed as V_S_[depth][action][next state]
            
            for a in range(self.A):
                for s in range(self.ns[a]):
                    if self.dones[a][s]:
                        V_S_[a][s] = self.done_value
            self.Q_S.append([(self.Q_S[0][a] + dep * V_S_[a])/(dep + 1) for a in range(self.A)])
            check.check("i")
            
            if self.advantage_learning:
                self.V_S__all.append([])
                i_a = [0 for _ in range(self.A)]
                for i in range(self.raw_S_.shape[0]):
                    a = self.actions[i]
                    elm = V_S_[a]
                    is_elm_empty = elm.numel() == 0
                    if is_elm_empty:
                        elm_to_append = torch.tensor([])
                    else:
                        elm_to_append = elm
                    self.V_S__all[-1].append(elm_to_append)
                    i_a[a] += 1
                self.V_S__all[-1] = torch.tensor(self.V_S__all[-1])

                self.V_S_estimate.append([[] for a in range(self.A)])
                V_S__estimate = [[] for a in range(self.A)]
                for i in range(self.raw_S.shape[0]):
                    V_s = (self.K_inv_S_S__all * self.k_batch(self.raw_S[i], self.raw_S_)) * (self.V_S__all[-1] - self.baseline) + self.baseline
                    V_s_ = (self.K_inv_S_S__all * self.k_batch(self.raw_S_[i], self.raw_S_)) * (self.V_S__all[-1] - self.baseline) + self.baseline
                    self.V_S_estimate[-1][self.actions[i]].append(V_s)
                    V_S__estimate[self.actions[i]].append(V_s_)
                self.V_S_estimate[-1] = [torch.tensor(arr) for arr in self.V_S_estimate[-1]]
                V_S__estimate = [torch.tensor(arr) for arr in V_S__estimate]
                if verbose and dep == 1:
                    fig = plt.figure()
                    ax = fig.add_subplot(1,1,1, projection='3d')
                    for a in range(self.A):
                        triang = mtri.Triangulation(self.S[a][:, 0],self.S[a][:, 1])
                        ax.plot_trisurf(triang, self.Q_S[-1][a] - self.V_S_estimate[-1][a], cmap='jet')
                        if a == 0:
                            color = "black"
                        if a == 1:
                            color = "red"
                        ax.scatter(self.S[a][:, 0],self.S[a][:, 1],self.Q_S[-1][a] - self.V_S_estimate[-1][a], marker='.', s=10, c=color, alpha=0.5)
                        ax.set_title("Q_S[1] - V_S_estimate[1] (on data)")
                    ax.view_init(elev=10, azim=-45)
                    plt.show()
                    plt.close()
            
            if dep < depth - 1:
                if self.advantage_learning:
                    self.Q_S_.append([[V_S__estimate[a] + self.w_SS_[a][a_] * (self.Q_S[-1][a_] - self.V_S_estimate[-1][a_])\
                                       if (self.w_SS_[a][a_] * (self.Q_S[-1][a_] - self.V_S_estimate[-1][a_])).numel() > 0 # What should [] * 5 be? or [] + 5? Current fix ignores cases when other tesnors are []
                                       else V_S__estimate[a]
                                       for a_ in range(self.A)]\
                                      for a in range(self.A)])
                else:
                    self.Q_S_.append([[self.baseline + self.w_SS_[a][a_] * (self.Q_S[-1][a_] - self.baseline)\
                                       for a_ in range(self.A)]\
                                      for a in range(self.A)])   
            check.check("j")         

    def estimate_Q(self, s, a, depth=-1):
        s = s.to(self.float_type)
        w = self.w(s, a)
        Q_S = self.Q_S[depth][a]
        if self.advantage_learning:
            # Replaced @ with *. Verify if torch.mm is required.
            V_s_estimate = (self.K_inv_S_S__all * self.k_batch(s, self.raw_S_)) * (self.V_S__all[depth] - self.baseline) + self.baseline
            return V_s_estimate + w * (Q_S - self.V_S_estimate[depth][a]) if w.numel() > 0 else V_s_estimate
        else:
            return self.baseline + w * (Q_S - self.baseline) if w.numel() > 0 else self.baseline

def k(s_1, s_2):
    max_var = 1/4
    return max_var * (1 - torch.mean(torch.abs(s_1 - s_2)))
    # return max_var * 2 ** (- 2 * torch.mean(torch.abs(s_1 - s_2))) # k(s, s) must = max_var

def k_batch(s_1, s_2s):
    max_var = 1/4
    return max_var * (1 - torch.mean(torch.abs(s_1.reshape(1, -1) - s_2s), axis=1))


In [17]:
# Tensors Version
class Tn_OptimisticAgent(object):
    def __init__(self, n_actions, state_space_kernel=k, 
                 state_space_batch_kernel=k_batch, optimism=3, 
                 horizon=40, done_value=0):
        self.A = n_actions
        self.k = state_space_kernel
        self.k_batch = state_space_batch_kernel
        self.optimism = optimism
        self.horizon = horizon
        self.done_value = done_value
        self.Q_estimator = None
        self.t = 0
    
    def observe(self, state, action, reward, next_state, done):
        if self.Q_estimator is None:
            self.Q_estimator = Tn_QueueQGP(
                self.A, self.k, self.k_batch, state.reshape(1, -1)
                , torch.tensor([action]), torch.tensor([reward]), next_state.reshape(1, -1)
                , torch.tensor([done]), done_value=self.done_value, optimistic=True, bias_count=self.optimism
                )
        else:
            self.Q_estimator.add_data_point(state, torch.tensor(action), torch.tensor(reward), next_state, done)
        self.Q_estimator.update_Q(self.horizon, verbose=self.t%20==19)

    
    def act(self, state):
        self.t += 1
        if self.Q_estimator is None:
            return 0
        t = torch.tensor(([self.Q_estimator.estimate_Q(state, a)\
                                          for a in range(self.A)
                                          ]))
        return torch.argmax(t)


In [18]:
estimator = Tn_QueueQGP(2, k, k_batch, torch.tensor([[0, 0]]), 
                        torch.tensor([0]), torch.tensor([0.5]), 
                        torch.tensor([[0.1, 0.1]]), torch.tensor([False]), 
                        done_value=0, optimistic=False, bias_count=1, 
                        advantage_learning=False)
estimator.update_Q(4)
for a in [0, 1]:
    print(estimator.estimate_Q(torch.tensor([0.1, 0.1]), a))
for i in range(1, 10):
    estimator.add_data_point(i*torch.tensor([0.1, 0.1]), torch.tensor(i//2 % 2), 
                             torch.tensor(0.5), (i+1)*torch.tensor([0.1, 0.1]), False)
    estimator.update_Q(4)
    for a in [0, 1]: 
        print(estimator.estimate_Q((i+1)*torch.tensor([0.1, 0.1]), a))



[1, 0]
els adv learning
tensor([], size=(1, 0))
0
self.K_SS_[a__][a] [tensor([[0.2250]])] K_inv_K_Ss tensor([0.4500])


TypeError: ignored

In [ ]:
agent = Tn_OptimisticAgent(2, horizon=5)
agent.observe(state=torch.tensor([0, 0]), 
              action=torch.tensor([0]), 
              reward=torch.tensor([0.1]), 
              next_state=torch.tensor([1, 0]), 
              done=False)

a = agent.act(torch.tensor([1, 0]))
print(a)

print(([agent.Q_estimator.estimate_Q(torch.tensor([1, 0]), a, depth=-1) for a in range(2)]))
agent.observe(
    state=torch.tensor([1, 0]), 
    action=a,
    reward=0.1, 
    next_state=torch.tensor([0, 0]),
    done=False)
a = agent.act(torch.tensor([0, 0]))
print(a)
